# ML

Import necessary libraries

In [ ]:
import numpy as np
from scipy.linalg import cholesky as chol, cho_solve

from joblib import Parallel, delayed
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

Some auxiliary functions

In [ ]:
def generar_grilla(sqrt_n):
    xx = np.linspace(1,sqrt_n,sqrt_n)
    X, Y = np.meshgrid(xx,xx)
    return np.column_stack((X.flatten(), Y.flatten())) #Ordenados de izq a der y de abajo hacia arriba

P = lambda alpha: np.array([[np.cos(alpha), np.sin(alpha)],[-np.sin(alpha), np.cos(alpha)]])
D = lambda ratio: np.diag([1,ratio])
A = lambda alpha, ratio: P(alpha) @ D(ratio) @ P(alpha).T

matern_model = lambda t, x: (1 + x/t) * np.exp(-x/t) # nu=3/2

def cholesky(alpha, t, ratio, H):
    sqrt_H_TAH = np.sqrt(np.einsum('ijk,ijk->ij', H @ A(alpha, ratio) , H))
    sigma = matern_model(t, sqrt_H_TAH)
    return chol(sigma, lower=True, overwrite_a=True) # np.linalg.cholesky(sigma)

def ML(alpha, t, ratio, H, Z):
    L = cholesky(alpha, t, ratio, H)
    log_det_cov = 2 * np.sum(np.log(np.diag(L)))
    temp = cho_solve((L, True), Z)
    return -0.5 * (log_det_cov + np.dot(Z, temp))

Get differences tensor

In [ ]:
sqrt_n = 16
points = generar_grilla(sqrt_n)
H = points[:, np.newaxis, :] - points[np.newaxis, :, :]

Download test set

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "..."

In [ ]:
X_test = np.load(path + 'X_test.npy')

## MLE with ``minimize``

In [ ]:
from scipy.optimize import minimize

def negative_log_likelihood(params, H, Z):
    alpha, t, ratio = params
    return -ML(alpha, t, ratio, H, Z)

In [ ]:
initial_guess = [np.deg2rad((0+180)/2), (0.02+5)/2, (0+1)/2]
tol = 1e-3
bounds = [(0, np.pi-tol), (0.0+tol, np.inf), (0.0+tol, 1.0)]

y_pred_ML_opt = []
X_test.shape

(115600, 16, 16)

In [ ]:
%%time
for field in tqdm(X_test, desc='progress', ncols=100, leave=False):
    Z = np.flipud(field).ravel()
    y_pred_ML_opt.append(minimize(negative_log_likelihood, initial_guess, args=(H, Z), bounds=bounds, method='L-BFGS-B').x)

CPU times: user 9h 2min 49s, sys: 6h 59min 12s, total: 16h 2min 2s
Wall time: 10h 25min 29s


In [ ]:
np.save(path+'y_pred_ML.npy', np.array(y_pred_ML_opt))

In [ ]:
len(y_pred_ML_opt)

115600